Query history

•𝑞0= the weather forecast in Koblenz

•𝑞1= next week weather forecast

•𝑞2= the weather tomorrow

•𝑞3= weather forecast Amsterdam

•𝑞4= the weather tomorrow in

Possible auto-complete outcomes i.e., words following ”the weather” in the query history 𝑎𝑖:

                                 •𝑎1=the weather the
                                 •𝑎2=the weather weather
                                 •𝑎3=the weather forecast
                                 •𝑎4=the weather in
                                 •𝑎5=the weather Koblenz
                                 •𝑎6=the weather next
                                 •𝑎7=the weather week
                                 •𝑎8=the weather tomorrow
                                 •𝑎9=the weather Amsterdam
                                 
Let 𝒟 be the corpus of all queries from the history

Vocab = {the weather forecast in koblenz next week tomorrow Amsterdam}

## Bigram Model
𝑃bi(𝑡1 𝑡2 𝑡3) =𝑃(𝑡1)𝑃(𝑡2/𝑡1)𝑃(𝑡3/𝑡2)

𝒫𝑏𝑖(the weather forecast)

=𝒫𝑢𝑛𝑖(𝑡ℎ𝑒)×𝒫𝑏𝑖(𝑤𝑒𝑎𝑡ℎ𝑒𝑟|𝑡ℎ𝑒)×𝒫𝑏𝑖(𝑓𝑜𝑟𝑒𝑐𝑎𝑠𝑡|𝑤𝑒𝑎𝑡ℎ𝑒𝑟)

=𝑡𝑓(𝑡ℎ𝑒,𝐷)/𝑑𝑙(𝐷) × 𝑡𝑓((𝑡ℎ𝑒,𝑤𝑒𝑎𝑡ℎ𝑒𝑟),𝐷)/𝑑𝑙(𝑡ℎ𝑒,𝐷) × 𝑡𝑓((𝑤𝑒𝑎𝑡ℎ𝑒𝑟,𝑓𝑜𝑟𝑒𝑐𝑎𝑠𝑡), 𝐷)/𝑑𝑙(𝑤𝑒𝑎𝑡ℎ𝑒𝑟,𝐷)

## Trigram Model

𝑃tri(𝑡1 𝑡2 𝑡3) =𝑃(𝑡1)𝑃(𝑡2/𝑡1)𝑃(𝑡3/𝑡1, 𝑡2)

𝒫𝑡𝑟𝑖(the weather tomorrow)

= 𝒫𝑢𝑛𝑖(𝑡ℎ𝑒)×𝒫𝑏𝑖(𝑤𝑒𝑎𝑡ℎ𝑒𝑟|𝑡ℎ𝑒)×𝒫𝑡𝑟𝑖(𝑡𝑜𝑚𝑜𝑟𝑟𝑜𝑤|𝑡ℎ𝑒𝑤𝑒𝑎𝑡ℎ𝑒𝑟)

=𝑡𝑓(𝑡ℎ𝑒,𝐷)/𝑑𝑙(𝐷) × 𝑡𝑓((𝑡ℎ𝑒,𝑤𝑒𝑎𝑡ℎ𝑒𝑟),𝐷)/𝑑𝑙(𝑡ℎ𝑒,𝐷) × 𝑡𝑓((𝑡ℎ𝑒,𝑤𝑒𝑎𝑡ℎ𝑒𝑟,𝑡𝑜𝑚𝑜𝑟𝑟𝑜𝑤),𝐷)/𝑑𝑙((the weather),𝐷)

In [1]:
import pandas as pd
import operator

In [2]:
D1='vaccine research corona virus research'
D2='research research cancer vaccine vaccine'
D3= 'virus virus corona vaccine lab'
D4= 'cancer lab research lab'

#q = 'vaccine vaccine research cancer'
corpus=[D1,D2,D3,D4]

In [3]:
D1.split()

['vaccine', 'research', 'corona', 'virus', 'research']

In [4]:
word_bag= set(D1.split()+D2.split()+D3.split()+D4.split())

In [5]:
word_bag

{'cancer', 'corona', 'lab', 'research', 'vaccine', 'virus'}

In [6]:
#𝑇 is the number of tokens in the whole collection
T= len(D1.split())+len(D2.split())+len(D3.split())+len(D4.split())

In [7]:
T

19

### 1-calcualting the probability of a term appearing in the whole corpus
## ${P_{M}}_c(t)= \frac{cf(t))}{T}$

In [8]:
#P_Mc is the probability of a term appearing in the whole corpus
cf={} # is for counting the corpus frequency
P_Mc={}
for term in word_bag:
    for doc in corpus: ##here I am counting the number of each term in whole corpus
        if term in cf.keys():
            cf[term]+=doc.count(term) 
        else:
            cf[term]= doc.count(term)
           
    P_Mc[term]=cf[term]/T # getting the probability of term by dividing it to the number of tokens in the whole collection

P_Mc

{'corona': 0.10526315789473684,
 'cancer': 0.10526315789473684,
 'virus': 0.15789473684210525,
 'vaccine': 0.21052631578947367,
 'research': 0.2631578947368421,
 'lab': 0.15789473684210525}

In [9]:
(sorted(P_Mc.values()))

[0.10526315789473684,
 0.10526315789473684,
 0.15789473684210525,
 0.15789473684210525,
 0.21052631578947367,
 0.2631578947368421]

### 2- Calcualting the probability of a term 𝑡 occurring in document 𝑑
## ${P_{M}}_d(t)= \frac{tf(t,d))}{dl}$

In [10]:
D1

'vaccine research corona virus research'

In [11]:
#P_Md is a dataframe for the probability of term 𝑡 occurring in document 𝑑

P_Md=pd.DataFrame(index=list(word_bag),columns=['D1','D2','D3','D4'])

#each doc is a column and each index is a term 
for term in word_bag:
    for doc,col in zip(corpus,P_Md.columns):
               
        P_Md[col][term]= doc.count(term)
P_Md

,D1,D2,D3,D4
corona,1,0,1,0
cancer,0,1,0,1
virus,1,0,2,0
vaccine,1,2,1,0
research,2,2,0,1
lab,0,0,1,2


In [12]:
for term in word_bag:
    for doc,col in zip(corpus,P_Md.columns):       
        P_Md[col][term]= doc.count(term)/len(doc.split()) #probability of each term in a doc is number of times term 
                                                            #occured divided by document length
P_Md

,D1,D2,D3,D4
corona,0.2,0.0,0.2,0.0
cancer,0.0,0.2,0.0,0.25
virus,0.2,0.0,0.4,0.0
vaccine,0.2,0.4,0.2,0.0
research,0.4,0.4,0.0,0.25
lab,0.0,0.0,0.2,0.5


In [13]:
P_Md = P_Md.sort_index()
P_Md

,D1,D2,D3,D4
cancer,0.0,0.2,0.0,0.25
corona,0.2,0.0,0.2,0.0
lab,0.0,0.0,0.2,0.5
research,0.4,0.4,0.0,0.25
vaccine,0.2,0.4,0.2,0.0
virus,0.2,0.0,0.4,0.0


In [14]:
P_Md.loc['vaccine'].values

array([0.2, 0.4, 0.2, 0.0], dtype=object)

### 3- Calculating the ranked result set according to the unsmoothed, uniform model
## $P_{uni}(d|q)=\prod_{t\varepsilon q}^{}{P_{M}}_d(t)$

In [15]:
q = 'vaccine vaccine research cancer'

In [16]:
#P_uni is the result set according to the unsmoothed, uniform model
P_uni={}

for col in P_Md.columns:
    P_uni[col]=1
    for term in q.split():  
      
        P_uni[col]= P_Md[col][term]*P_uni[col] #the unigram model is the multiplication of unigram probability of terms(in q) 
                                                ##that we found them in the previous cell

print(P_uni,"\n\n")
sorted_P_uni = sorted(P_uni.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_P_uni)

# P(D1/q) = 0.2*0.2*0.4*0 = 0 
# P(D2/q) = 0.4*0.4*0.4*0.2 = 0.0128
# P(D3/q) = 0.2*0.2*0*0 = 0
# P(D4/q) = 0*0*0.25*0.25 =0

# Rank D2> (D1=D3=D4)

{'D1': 0.0, 'D2': 0.012800000000000004, 'D3': 0.0, 'D4': 0.0} 


[('D2', 0.012800000000000004), ('D1', 0.0), ('D3', 0.0), ('D4', 0.0)]


In [17]:
#P_interp is the result set according to the linear-interpolated, uniform model
P_interp={}
lamb=0.5

for col in P_Md.columns:
    P_interp[col]=1
    for term in q.split():      
        P_interp[col]= ((lamb*P_Md[col][term])+((1-lamb)*P_Mc[term]))*P_interp[col]
        
P_interp

{'D1': 0.0007352844131030303,
 'D2': 0.004716068784002579,
 'D3': 0.000291779529009139,
 'D4': 0.0005050030309773557}

In [18]:
sorted_P_interp = sorted(P_interp.items(), key=operator.itemgetter(1), reverse=True)
sorted_P_interp

[('D2', 0.004716068784002579),
 ('D1', 0.0007352844131030303),
 ('D4', 0.0005050030309773557),
 ('D3', 0.000291779529009139)]

In [19]:
[round(i,4) for i in (sorted(P_interp.values(), reverse=True))]

[0.0047, 0.0007, 0.0005, 0.0003]

In [20]:
P_Mc

{'corona': 0.10526315789473684,
 'cancer': 0.10526315789473684,
 'virus': 0.15789473684210525,
 'vaccine': 0.21052631578947367,
 'research': 0.2631578947368421,
 'lab': 0.15789473684210525}

In [21]:
P_Md

,D1,D2,D3,D4
cancer,0.0,0.2,0.0,0.25
corona,0.2,0.0,0.2,0.0
lab,0.0,0.0,0.2,0.5
research,0.4,0.4,0.0,0.25
vaccine,0.2,0.4,0.2,0.0
virus,0.2,0.0,0.4,0.0


In [22]:
q='vaccine research'

In [23]:
P_uni={}

for col in P_Md.columns:
    P_uni[col]=1
    for term in q.split():  
      
        P_uni[col]= P_Md[col][term]*P_uni[col] #the unigram model is the multiplication of unigram probability of terms(in q) 
                                                ##that we found them in the previous cell

print(P_uni,"\n\n")
sorted_P_uni = sorted(P_uni.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_P_uni)

{'D1': 0.08000000000000002, 'D2': 0.16000000000000003, 'D3': 0.0, 'D4': 0.0} 


[('D2', 0.16000000000000003), ('D1', 0.08000000000000002), ('D3', 0.0), ('D4', 0.0)]


In [24]:
P_interp={}
lamb=0.5

for col in P_Md.columns:
    P_interp[col]=1
    for term in q.split():      
        P_interp[col]= ((lamb*P_Md[col][term])+((1-lamb)*P_Mc[term]))*P_interp[col]
        
P_interp

{'D1': 0.06806094182825485,
 'D2': 0.10121883656509696,
 'D3': 0.027008310249307478,
 'D4': 0.027008310249307475}

In [25]:
sorted_P_interp = sorted(P_interp.items(), key=operator.itemgetter(1), reverse=True)
sorted_P_interp

[('D2', 0.10121883656509696),
 ('D1', 0.06806094182825485),
 ('D3', 0.027008310249307478),
 ('D4', 0.027008310249307475)]

In [26]:
[round(i,4) for i in (sorted(P_interp.values()))]

[0.027, 0.027, 0.0681, 0.1012]